In [1]:
import pandas as pd
import matplotlib.colors as pltcolors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import gridspec
from datetime import datetime
import ipywidgets as widgets
import copy
from datetime import datetime
import random

#Lectura de identificadores
#Como los archivos descargados siguen un formato que involucra el identificador, es util tener a mano estos datos para agrupar los datos

f = open("ID_Estaciones.txt", "r")
ids = []
lines = f.readlines()
ids_dict = {}
estaciones = {}

for line in lines:
    temp = line.split(",")
    ids.append(line.split(",")[len(temp) - 1].split("\n")[0])
    ids_dict[line.split(",")[len(temp) - 1].split("\n")[0]] = {}
    estaciones[line.split(",")[len(temp) - 1].split("\n")[0]] = temp

print(ids)
print(ids_dict)
labels = copy.deepcopy(ids)

['510005', '400009', '180005', '330020', '550001', '270008', '290004', '410005', '430002', '430009', '450001', '450005', '530005', '200006', '330113']
{'510005': {}, '400009': {}, '180005': {}, '330020': {}, '550001': {}, '270008': {}, '290004': {}, '410005': {}, '430002': {}, '430009': {}, '450001': {}, '450005': {}, '530005': {}, '200006': {}, '330113': {}}


In [2]:
#Parte 0 del modelo: Lectura de archivos y agregacion  de datos.
#
#Se reemplazaran los datos NaN y añadira la eficiencia de los paneles solares al dataset de paneles solares.

bigDf = pd.DataFrame()
for id in ids:
    df = pd.read_excel(id+"_agrupado.xlsx", sheet_name="Sheet1")
    if(bigDf.size == 0):
        bigDf = pd.concat([df.iloc[:,1:]], ignore_index = True)
    else:
        bigDf = pd.concat([bigDf,df.iloc[:,1:]], ignore_index = True)

bigDf["momento"] = pd.to_datetime(bigDf["momento"], format="%d-%m-%Y %H:%M:%S")
bigDf["Nombre_Estacion"] = ""
bigDf["Region"] = ""
for key,value in estaciones.items():
    bigDf.loc[bigDf["ID_Estacion"] == int(key), "Nombre_Estacion"] = value[0]
    bigDf.loc[bigDf["ID_Estacion"] == int(key), "Region"] = value[1]

bigDf= bigDf.fillna(0)

bigDf

solar = pd.DataFrame()
df = pd.read_excel("Paneles_Solartex.xlsx", sheet_name="Hoja1")

if(solar.size == 0):
    solar = pd.concat([df.iloc[:,0:]], ignore_index = True)
else:
    solar = pd.concat([solar,df.iloc[:,0:]], ignore_index = True)

ef = (solar.iloc[:,1] / (1000*((solar.iloc[:,3]/1000) * (solar.iloc[:,2]/1000))))*100

solar.insert(2,"Eficiencia(%)",ef,True)

aero = pd.DataFrame()
df = pd.read_excel("Paneles_Solartex.xlsx", sheet_name="Hoja3")
if(aero.size == 0):
    aero = pd.concat([df.iloc[:,0:]], ignore_index = True)
else:
    aero = pd.concat([aero,df.iloc[:,0:]], ignore_index = True)


In [3]:
#Parte 1 del modelo: Calculando la potencia "teorica", la potencia "real" que generan los dispositivos y catalogamiento del registro
#
#Por potencia "teorica" se refiere a la potencia generada solo considerando el viento/radiacion registrada, sin factores adicionales
#Por potencia "real" se refiere a la potencia generada considerando factores adicionales. Estos factores adicionales afectan en la potencia total resultante
#La potencia real generada se calcula a traves de escalares que determinan el porcentaje de impacto de cada factor.
#Los porcentajes son calculados a traves de un mapeo de los valores hasta los porcentajes maximos estimados que podrian afectar.
#Por "catalogamiento" se refiere al proceso de etiquetar un registro del dataset con un valor que diga si las condiciones registradas son viables o no para instalar un panel solar/aerogenerador
#El criterio para decidir si es viable o no es definido por si el dispositivo es capaz de generar potencial por sobre un porcentaje del maximo estipulado en sus especificaciones:
#Ej1: Si el dispositivo x genera potencial por sobre el 50% del maximo estipulado, es viable, por lo tanto se catalogara como "si". En caso contrario, se cataloga con "no"
#
#Los valores resultantes luego son adjuntados al dataset principal


for x in range(solar.shape[0]):
    temp = solar.iloc[x,:]
    sunDf = pd.DataFrame()

    sunDf["potencia"] = ((temp["Ancho(mm)"]/1000) * (temp["Alto(mm)"]/1000)) * bigDf["RadGInst (watt/m2)"] * (temp["Eficiencia(%)"]/100)
    sunDf["lo vale?"] = "no"

    m = [-1, -1, 0, 0, 0, 1]
    frag = bigDf.iloc[:,2:7]
    frag["potencia_t"] = sunDf["potencia"]
    
    frag["HR_Valor(%)"] = (frag["HR_Valor(%)"] - 60) / (100 - 60)
    frag["HR_Valor(%)"] = 0 + (frag["HR_Valor(%)"]  * (0.01 - 0))
    frag.loc[frag["HR_Valor(%)"] < 0, "HR_Valor(%)"] = 0

    frag["RR6_Valor(mm)"] = (frag["RR6_Valor(mm)"] - 0) / (max(frag["RR6_Valor(mm)"]))
    frag["RR6_Valor(mm)"] = 0 + (frag["RR6_Valor(mm)"]  * 0.75)

    frag["potencia_r"] = ((-1)*frag["potencia_t"].mul(frag["HR_Valor(%)"]) +
                            (-1)*frag["potencia_t"].mul(frag["RR6_Valor(mm)"]) +
                            frag["potencia_t"])
    
    sunDf.loc[frag["potencia_r"] >= (temp["Potencia Maxima(W)"]/2), "lo vale?"] = "si"
    bigDf.insert(bigDf.shape[1],"Panel Solar "+str(x),frag["potencia_r"],True)
    bigDf.insert(bigDf.shape[1],"Panel Solar "+str(x)+"_dec",sunDf["lo vale?"],True)


def translateWind(value, fromMin, fromMax, toMin, toMax, clause):
    tmp = value["ff_Valor(kt)"] * 0.514444
    # Figure out how 'wide' each range is
    tmp = pd.DataFrame(tmp)
    tmp["potencia"] = 0
    tmp["lo vale?"] = "no"
    #display(tmp)
    fromSpan = fromMax - fromMin
    toSpan = toMax - toMin

    # Convert the left range into a 0-1 range (float)
    tmp["potencia"] = (tmp["ff_Valor(kt)"] - fromMin) / (fromSpan)
    tmp["potencia"] = toMin + (tmp["potencia"]  * toSpan)

    tmp.loc[tmp["ff_Valor(kt)"] < fromMin, "potencia"] = 0
    tmp.loc[tmp["ff_Valor(kt)"] >= fromMax, "potencia"] = toMax
    tmp.loc[tmp["ff_Valor(kt)"] >= clause, "potencia"] = 0


    frag = value.iloc[:,2:7]
    frag["potencia_t"] = tmp["potencia"]
    m = [-1, -1, -1, 0, 0, 1]
    frag["HR_Valor(%)"] = (frag["HR_Valor(%)"] - 50) / (100 - 50)
    frag["HR_Valor(%)"] = 0 + (frag["HR_Valor(%)"]  * (0.05 - 0))
    frag.loc[frag["HR_Valor(%)"] < 0, "HR_Valor(%)"] = 0

    frag["RR6_Valor(mm)"] = (frag["RR6_Valor(mm)"] - 0) / (max(frag["RR6_Valor(mm)"]))
    frag["RR6_Valor(mm)"] = 0 + (frag["RR6_Valor(mm)"]  * 0.005)
    
    frag["Ts_Valor(°C)"] = (frag["Ts_Valor(°C)"] - 0) / (40)
    frag["Ts_Valor(°C)"] = 0 + (frag["Ts_Valor(°C)"]  * 0.01)

    frag["potencia_r"] = ((-1)*frag["potencia_t"].mul(frag["HR_Valor(%)"]) +
                            (-1)*frag["potencia_t"].mul(frag["Ts_Valor(°C)"]) +
                            (-1)*frag["potencia_t"].mul(frag["RR6_Valor(mm)"]) +
                            frag["potencia_t"])
    #display(frag)
    tmp["potencia"] = frag["potencia_r"]
    tmp.loc[frag["potencia_r"] >= (toMax/2), "lo vale?"] = "si"

    # Convert the 0-1 range into a value in the right range.
    return tmp

for x in range(aero.shape[0]):
    temp = aero.iloc[x,:]
    #display(temp)
    a = translateWind(bigDf,temp["Viento de arranque(m/s)"], temp["Viento nominal(m/s)"],0,temp["Potencia nominal(W)"],temp["Viento de corte(m/s)"])
    bigDf.insert(bigDf.shape[1],"Aerogenerador "+str(x),a["potencia"],True)
    bigDf.insert(bigDf.shape[1],"Aerogenerador "+str(x)+"_dec",a["lo vale?"],True)
bigDf["ff_Valor(kt)"] = bigDf["ff_Valor(kt)"] * 0.514444
bigDf.rename(columns={"ff_Valor(kt)":"ff_Valor(m/s)"},inplace=True)

In [4]:
#Paso 2 del modelo: Decision de viabilidad en una estacion. Luego de catalogar todos los registros, se contara la cantidad de "si" y "no"
#y se decidira si es viable instalar un dispositivo cerca de las estaciones determinadas.
#La decision se basa en si la cantidad de "si" supera un porcentaje del total de los registros de la estacion.
#Por ej: La cantidad de registros en la estacion A es de 8760, y la cantidad de registros etiquetados con "si" en esta estacion es de 1390.
#        Para decir que es viable, la cantidad de "si" debe superar el 15% del total de registros de la estacion A:
#        8760*0.15 = 1314. Como 1390 es mayor a 1314, se puede decir que la instalacion del dispositivo es viable desde el punto de vista energetico.
#
#El motivo de estos porcentajes se debe a:
#1) La energia solar estara siempre condicionada por la luz del dia. Como los registros de las estaciones estan etiquetados con FECHA/HORA e idealmente,
#los registros son de un año entero, se asume que hay una porcion de estos registros que es del dia, mientras que la otra porcion es de noche. 
#Al seleccionar un porcentaje del total de los registros, se intenta apuntar a que dicho porcentaje sea de las horas con luz del dia.
#Esto se traduce a que se necesita que los registros etiquetados con "si" sean aproximadamente un octavo o más(12.5% o 15%) del total de registros de una estacion.
#2) En el caso de los aerogeneradores, como esta solo condicionado por el viento, es más flexible, pero a la vez más inconsistente.
#Esto significa que se necesita una cantidad mucho más grande de registros para decir que la generacion de energia a traves de aerogeneradores es consistente
#Lo que se traduce a que se necesita que la cantidad de registros etiquetados con "si", debe ser mayor, idealmente cercana a la mitad(40% o 50%) del total de registros de una estacion.
#
#La decision de si es finalmente viable o no instalar un dispositivo en un sector cercano a las estaciones, es reflejada en dos datasets,
# uno para paneles solares y otro para aerogeneradores, cuyos contenidos son:
# - el total de registros 
# - el total de etiquetas
# - la potencia "real" generada
# - la etiqueta viabilidad, denotando con un 1 si es viable, o un 0 en caso contrario
# - las filas estan etiquetadas por el ID de la estacion
# - las columnas estan etiquetadas con los alias de los dispositivos(Ej: PS_x(Panel solar) o AERO_x(Aerogenerador))

countDf1 = pd.DataFrame()
for i in ids:
    preDf = pd.DataFrame()
    q = bigDf.query('ID_Estacion == '+i)
    for x in range(solar.shape[0]):
        preDf["PS_"+str(x)] = q["Panel Solar "+str(x)+"_dec"].value_counts()
        preDf.loc["total_decisiones","PS_"+str(x)] = q["Panel Solar "+str(x)+"_dec"].count()
        preDf.loc["potencia_total(W)","PS_"+str(x)] = q["Panel Solar "+str(x)].sum()
        if(preDf.loc["no","PS_"+str(x)] == preDf.loc["total_decisiones","PS_"+str(x)]):
            preDf.loc["si","PS_"+str(x)] = 0
        if(preDf.loc["si","PS_"+str(x)] >= preDf.loc["total_decisiones","PS_"+str(x)]*0.15):
            preDf.loc["veredicto","PS_"+str(x)] = 1
        else:
            preDf.loc["veredicto","PS_"+str(x)] = 0
    preDf = pd.concat({i: preDf}, names=['estacion'])
    countDf1 = pd.concat([countDf1, preDf], axis=0)

countDf2 = pd.DataFrame()
for i in ids:
    preDf = pd.DataFrame()
    q = bigDf.query('ID_Estacion == '+i)
    for y in range(aero.shape[0]):
        preDf["AERO_"+str(y)] = q["Aerogenerador "+str(y)+"_dec"].value_counts()
        preDf.loc["total_decisiones","AERO_"+str(y)] = q["Aerogenerador "+str(y)+"_dec"].count()
        preDf.loc["potencia_total(w)","AERO_"+str(y)] = q["Aerogenerador "+str(y)].sum()
        if(preDf.loc["no","AERO_"+str(y)] == preDf.loc["total_decisiones","AERO_"+str(y)]):
            preDf.loc["si","AERO_"+str(y)] = 0
        if(preDf.loc["si","AERO_"+str(y)] >= preDf.loc["total_decisiones","AERO_"+str(y)]*0.4):
            preDf.loc["veredicto","AERO_"+str(y)] = 1
        else:
            preDf.loc["veredicto","AERO_"+str(y)] = 0
    preDf = pd.concat({i: preDf}, names=['estacion'])
    countDf2 = pd.concat([countDf2, preDf], axis=0)

pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

countDf1 = countDf1.rename_axis(["estacion", "info"])
countDf2 = countDf2.rename_axis(["estacion", "info"])


In [5]:
countDf1

PS_0  PS_1  PS_2  PS_3   PS_4   PS_5   PS_6  \
estacion info                                                             
510005   no                 8037  8037  8037  8037   8037   8037   8037   
         si                  762   762   762   762    762    762    762   
         total_decisiones   8799  8799  8799  8799   8799   8799   8799   
         potencia_total(W) 11495 22990 28738 34485  57475 114950 114950   
         veredicto             0     0     0     0      0      0      0   
400009   no                 7605  7605  7605  7605   7605   7605   7605   
         si                  931   931   931   931    931    931    931   
         total_decisiones   8536  8536  8536  8536   8536   8536   8536   
         potencia_total(W) 12566 25131 31414 37697  62828 125655 125655   
         veredicto             0     0     0     0      0      0      0   
180005   no                 6833  6833  6833  6833   6833   6833   6833   
         si                 1940  1940  1940  1940   1940   1940   1940   
         total_decisiones   8773  8773  8773  8773   8773   8773   8773   
         potencia_total(W) 20391 40781 50977 61172 101953 203907 203907   
         veredicto             1     1     1     1      1      1      1   
330020   no                 7181  7181  7181  7181   7181   7181   7181   
         si                 1578  1578  1578  1578   1578   1578   1578   
         total_decisiones   8759  8759  8759  8759   8759   8759   8759   
         potencia_total(W) 17373 34746 43432 52119  86864 173728 173728   
         veredicto             1     1     1     1      1      1      1   
550001   no                 7872  7872  7872  7872   7872   7872   7872   
         si                  507   507   507   507    507    507    507   
         total_decisiones   8379  8379  8379  8379   8379   8379   8379   
         potencia_total(W)  9005 18009 22512 27014  45023  90047  90047   
         veredicto             0     0     0     0      0      0      0   
270008   no                 7118  7118  7119  7118   7118   7118   7118   
         si                 1829  1829  1828  1829   1829   1829   1829   
         total_decisiones   8947  8947  8947  8947   8947   8947   8947   
         potencia_total(W) 19697 39394 49242 59090  98484 196968 196968   
         veredicto             1     1     1     1      1      1      1   
290004   no                 7282  7282  7282  7282   7282   7282   7282   
         si                 1605  1605  1605  1605   1605   1605   1605   
         total_decisiones   8887  8887  8887  8887   8887   8887   8887   
         potencia_total(W) 17541 35082 43852 52623  87705 175409 175409   
         veredicto             1     1     1     1      1      1      1   
410005   no                 7898  7898  7898  7898   7898   7898   7898   
         si                  871   871   871   871    871    871    871   
         total_decisiones   8769  8769  8769  8769   8769   8769   8769   
         potencia_total(W) 12611 25223 31528 37834  63057 126113 126113   
         veredicto             0     0     0     0      0      0      0   
430002   no                 7529  7529  7529  7529   7529   7529   7529   
         si                 1225  1225  1225  1225   1225   1225   1225   
         total_decisiones   8754  8754  8754  8754   8754   8754   8754   
         potencia_total(W) 14757 29514 36892 44271  73785 147570 147570   
         veredicto             0     0     0     0      0      0      0   
430009   no                 7641  7641  7641  7641   7641   7641   7641   
         si                  877   877   877   877    877    877    877   
         total_decisiones   8518  8518  8518  8518   8518   8518   8518   
         potencia_total(W) 12277 24554 30693 36832  61386 122772 122772   
         veredicto             0     0     0     0      0      0      0   
450001   no                 6911  6911  6911  6911   6911   6911   6911   
         si                  671   671   671   671   

In [6]:
countDf2

AERO_0  AERO_1  AERO_2  AERO_3  AERO_4  AERO_5  \
estacion info                                                                
510005   no                   6310    6022    6316    6310    5703    6310   
         si                   2489    2777    2483    2489    3096    2489   
         total_decisiones     8799    8799    8799    8799    8799    8799   
         potencia_total(w) 1104817 1172866 1083712 1657226 3089096 1657226   
         veredicto               0       0       0       0       0       0   
400009   no                   8389    8322    8408    8389    8159    8389   
         si                    147     214     128     147     377     147   
         total_decisiones     8536    8536    8536    8536    8536    8536   
         potencia_total(w)  261719  324835  295317  392579  667533  392579   
         veredicto               0       0       0       0       0       0   
180005   no                   8605    8434    8605    8605    8169    8605   
         si                    168     339     168     168     604     168   
         total_decisiones     8773    8773    8773    8773    8773    8773   
         potencia_total(w)  412994  497061  451874  619491 1096626  619491   
         veredicto               0       0       0       0       0       0   
330020   no                   8738    8726    8738    8738    8726    8738   
         si                     21      33      21      21      33      21   
         total_decisiones     8759    8759    8759    8759    8759    8759   
         potencia_total(w)   34881   52962   48147   52322   83330   52322   
         veredicto               0       0       0       0       0       0   
550001   no                   6915    6638    6932    6915    6247    6915   
         si                   1464    1741    1447    1464    2132    1464   
         total_decisiones     8379    8379    8379    8379    8379    8379   
         potencia_total(w)  795069  864816  790454 1192603 2255283 1192603   
         veredicto               0       0       0       0       0       0   
270008   no                   8774    8661    8788    8774    8455    8774   
         si                    173     286     159     173     492     173   
         total_decisiones     8947    8947    8947    8947    8947    8947   
         potencia_total(w)  455093  555872  505373  682640 1171116  682640   
         veredicto               0       0       0       0       0       0   
290004   no                   8823    8706    8823    8823    8446    8823   
         si                     64     181      64      64     441      64   
         total_decisiones     8887    8887    8887    8887    8887    8887   
         potencia_total(w)  390496  471444  428586  585743 1025150  585743   
         veredicto               0       0       0       0       0       0   
410005   no                   8136    8036    8204    8136    7784    8136   
         si                    633     733     565     633     985     633   
         total_decisiones     8769    8769    8769    8769    8769    8769   
         potencia_total(w)  510868  590528  537901  766303 1385497  766303   
         veredicto               0       0       0       0       0       0   
430002   no                   8135    7987    8136    8135    7804    8135   
         si                    619     767     618     619     950     619   
         total_decisiones     8754    8754    8754    8754    8754    8754   
         potencia_total(w)  438031  518148  471316  657046 1179847  657046   
         veredicto               0       0       0       0       0       0   
430009   no                   7617    7529    7740    7617    7175    7617   
         si                    901     989     778     901    1343     901   
         total_decisiones     8518    8518    8518    8518    8518    8518   
         potencia_total(w)  733879  834494  764428 1100819 1980468 1100819   
         veredicto               0       0       0

In [7]:
countDf1.query("info == 'veredicto'")

,,PS_0,PS_1,PS_2,PS_3,PS_4,PS_5,PS_6,PS_7,PS_8,PS_9,PS_10,PS_11,PS_12,PS_13,PS_14,PS_15,PS_16,PS_17,PS_18,PS_19,PS_20,PS_21,PS_22,PS_23,PS_24,PS_25,PS_26,PS_27,PS_28,PS_29,PS_30
estacion,info,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
510005,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
400009,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
180005,veredicto,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
330020,veredicto,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
550001,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
270008,veredicto,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
290004,veredicto,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
410005,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
430002,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
countDf2.query("info == 'veredicto'")

,,AERO_0,AERO_1,AERO_2,AERO_3,AERO_4,AERO_5,AERO_6,AERO_7,AERO_8,AERO_9,AERO_10,AERO_11,AERO_12,AERO_13
estacion,info,,,,,,,,,,,,,,
510005,veredicto,0,0,0,0,0,0,0,1,1,1,0,0,0,0
400009,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
180005,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
330020,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
550001,veredicto,0,0,0,0,0,0,0,1,0,0,0,0,0,0
270008,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
290004,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
410005,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
430002,veredicto,0,0,0,0,0,0,0,0,0,0,0,0,0,0
